# Experimento 5 
***
- Rede ResNet
- Conjunto de dados NIH
- Analisando o treinamento de uma rede com multiclassificadores

### Importação dos pacotes

In [1]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.applications.resnet import ResNet152
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

### Importação dos dados

In [2]:
# importando os dataframes dos dados de treinamento, validação e teste
train_df = pd.read_csv('/content/drive/MyDrive/train_df.csv', sep = ',', index_col=  0)
validation_df = pd.read_csv('/content/drive/MyDrive/validation_df.csv', sep = ',', index_col=  0)
test_df = pd.read_csv('/content/drive/MyDrive/test_df.csv' , sep = ',', index_col=  0)

In [3]:
# visualizando como é a organização do dataframe pré-processado
train_df.head()

,Image Index,finding_labels,labels
87745,/content/drive/MyDrive/images-nih1/images/0000...,Consolidation,1
82970,/content/drive/MyDrive/images-nih1/images/0000...,No Finding,0
51704,/content/drive/MyDrive/images-nih3/images/0001...,No Finding,0
38340,/content/drive/MyDrive/images-nih3/images/0001...,No Finding,0
71153,/content/drive/MyDrive/images-nih4/images/0002...,Infiltration,1


In [4]:
# visualizando a quantidade de imagens disponíveis por classe
train_df['finding_labels'].value_counts()

No Finding            55622
Infiltration           8770
Atelectasis            3902
Effusion               3645
Nodule                 2497
Pneumothorax           2024
Consolidation          1224
Pleural_Thickening     1028
Emphysema               818
Fibrosis                671
Edema                   578
Pneumonia               295
Name: finding_labels, dtype: int64

In [5]:
# organizando o balanceamento das classes
total_train_examples = 55622 + 8770 + 3902 + 3645 + 2497 + 2024 + 1224 + 1028 + 818 + 671 + 578 + 295
class_weight = {0: total_train_examples / 55622,
                1: total_train_examples / 8770,
                2: total_train_examples / 3902,
                3: total_train_examples / 3645,
                4: total_train_examples / 2497,
                5: total_train_examples / 2024,
                6: total_train_examples / 1224,
                7: total_train_examples / 1028,
                8: total_train_examples / 818,
                9: total_train_examples / 671,
                10: total_train_examples / 578,
                11: total_train_examples / 295}

In [6]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255., rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      x_col = 'Image Index',
                                                      y_col = 'finding_labels',
                                                      batch_size = 256,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      color_mode = 'rgb',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                    dataframe = validation_df,
                                                    x_col = 'Image Index',
                                                    y_col = 'finding_labels',
                                                    batch_size = 128,
                                                    seed = 42,
                                                    shuffle = True,
                                                    class_mode = 'categorical',
                                                    target_size = (256, 256))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  x_col = 'Image Index',
                                                  y_col = 'finding_labels',
                                                  batch_size = 128,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 81074 validated image filenames belonging to 12 classes.
Found 4268 validated image filenames belonging to 12 classes.
Found 2640 validated image filenames belonging to 12 classes.


### Preparando a rede neural convolucional

In [7]:
# realizando a transferência de aprendizagem com a rede ResNet
base_model = ResNet152(input_shape = (256, 256, 3), weights = 'imagenet', include_top = False)

# pegando a última camada da DenseNet
x = base_model.output
# adicionando uma camada de Global Average Pooling
x = layers.GlobalAveragePooling2D()(x)
# adicionando uma camada densa com 512 neurônios
x = layers.Dense(units = 512, activation = tf.nn.relu)(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 256, activation = tf.nn.relu)(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)
# adicionando a camada de saída da rede 
predictions = layers.Dense(units = 12, activation = tf.nn.softmax)(x)

# concatenando a rede importada com a rede definida (ajuste fino)
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# definindo as flags iniciais  
model.trainable = True
set_trainable = False

# para a arquitetura DenseNet, a rede será retreinada a partir da camada 'conv5_block1_1_conv'
for layer in model.layers:
    if layer.name == 'conv5_block1_1_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:
# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [10]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/resnet/transferlearning_weights_v2.0.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, 
                             mode = 'max')

In [11]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 81072 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 1, class_weight = class_weight,
                              use_multiprocessing = True, workers = 4)

 24/316 [=>............................] - ETA: 1:16:58 - loss: 112.7497 - acc: 0.5704